# dbt with Databricks - Hands-on Exercises

Welcome to the dbt training session! In this workshop, you'll learn to build data transformations using the medallion architecture pattern.

![Medallion Architecture Overview](z_assets/medallion-architecture-overview.png)

## What You'll Build
- **Bronze Layer**: Raw TPC-H data (already provided)
- **Silver Layer**: Cleansed data with surrogate keys and SCD2 tracking
- **Gold Layer**: Dimensional model for analytics

## Prerequisites
- Access to Databricks workspace

## Exercise 1: dbt Setup

### Goal
Initialize your dbt project and establish connection to Databricks.

### Steps

#### 1.1 Initialize virtual environment
```bash
python -m venv .venv

# Windows
.venv\Scripts\activate

# Mac/Linux
. .venv/bin/activate
```

#### 1.1 Initialize dbt Project
```bash
pip install -r requirements-dev.txt

dbt init
dbt debug
```

#### 1.2 Review Connection
Review `C:\Users\<Username>\.dbt\profiles.yml` (Windows) or `~/.dbt/profiles.yml` (Mac/Linux) and optionally copy the file to dbt-workshop:
```yaml
dbt-workshop:
  outputs:
    dev:
      catalog: dbt-workshop
      host: adb-673939630363416.16.azuredatabricks.net
      http_path: /sql/1.0/warehouses/42d4ebf47e760187
      schema: <YOUR INITIALS>
      threads: 4
      token: <PAT>
      type: databricks
  target: dev
```

#### 1.3 Test Connection
```bash
dbt debug
```

### Expected Outcome
- `dbt debug` shows all green checkmarks (except git)
- Connection to Databricks is successful

## Exercise 2: Explore Bronze Data

### Goal
Understand the TPC-H source data structure.

### TPC-H Data Model
![TPC-H Data Model](z_assets/tpc-h.png)

>> The listed columns are not correct. The correct column names have prefixes as seen in the paranthesis after the table name. E.g., `custkey` of the table **custoer** is actually `c_custkey`. You have to keep that in mind during the exercises.

### Steps

#### 2.1 Login to the Databricks Workspace
1. Go to: https://adb-673939630363416.16.azuredatabricks.net/
2. On the menu click on "SQL Editor"
3. Make sure that you have the catalog 'dbt-workshop' and the schema 'default' selected.
![image.png](z_assets/exercise-2-1.png)

#### 2.2 Explore Tables
Explore the tables in the catalog using the UI.

Run these queries in Databricks to understand the data:

```sql
-- Check available tables
SHOW TABLES IN bronze;

-- Explore customer data
SELECT * FROM bronze.customer LIMIT 10;
DESCRIBE bronze.customer;

-- Check data volumes
SELECT 'customer' as table_name, COUNT(*) as row_count FROM bronze.customer
UNION ALL
SELECT 'orders', COUNT(*) FROM bronze.orders
UNION ALL
SELECT 'lineitem', COUNT(*) FROM bronze.lineitem;
```

### Tables Available
- `customer`: Customer information
- `orders`: Customer orders 
- `lineitem`: Order line items (largest table)
- `part`: Product parts
- `partsupp`: Part-supplier relationships
- `supplier`: Supplier information
- `nation`: Countries
- `region`: Geographic regions

## Exercise 3: Silver Layer - Basic Models

### Goal
Create your first dbt models with renamed tables and columns.

### Steps

#### 3.1 Create Source Configuration
Create `src/models/bronze/sources.yml`:
```yaml
version: 2

sources:
  - name: bronze
    description: 'Raw TPC-H data'
    tables:
      - name: customer
      - name: orders
      - name: lineitem
      - name: part
      - name: partsupp
      - name: supplier
      - name: nation
      - name: region
```

#### 3.2 Configure Silver Models
Create `src/models/silver/schema.yml`:
```yaml
version: 2

models:
  - name: customers
    description: 'Cleansed customer data'
  - name: orders
    description: 'Cleansed orders data'
  - name: lineitems
    description: 'Cleansed line item data'
  - name: parts
    description: 'Cleansed parts data'
  - name: suppliers
    description: 'Cleansed supplier data'
  - name: nations
    description: 'Cleansed nation data'
  - name: regions
    description: 'Cleansed region data'
```

#### 3.3 Create Silver Models

Create `src/models/silver/customers.sql`:
```sql
{{ config(materialized='table') }}

SELECT 
    c_custkey as customer_id,
    c_name as customer_name,
    c_address as customer_address,
    c_nationkey as nation_id,
    c_phone as customer_phone,
    c_acctbal as account_balance,
    c_mktsegment as market_segment,
    c_comment as customer_comment
FROM {{ source('bronze', 'customer') }}
```

You don't have to specify `{{ config(materialized='table') }}` within the model, as we have already a project-wide setting for table materialization specified in `dbt_project.yml`.

Create similar models for:
- `orders.sql` (rename orderkey → order_id, custkey → customer_id, etc.)
- `lineitems.sql` (rename orderkey → order_id, partkey → part_id, etc.)
- `parts.sql`, `suppliers.sql`, `nations.sql`, `regions.sql`
- Rules:
  - Table names in silver are plural
  - If it is a comment field, then add the entity name as prefix: `c_customer` → `customer_comment`.
  - Abbreviations spelled out: `qty` → `quantity`, etc.
  - snake_case for columns containing multiple words
- You can skip `partsupp` as this table is not required for our use case

#### 3.4 Run Your Models
```bash
dbt run --select silver

# Run all models
dbt run
```

### Success Criteria
- All silver models run successfully
- Tables have consistent naming (plural, snake_case)
- Columns are renamed for clarity

## Exercise 4: Silver Layer - Surrogate Keys & Snapshots

### Goal
Add surrogate keys and implement SCD2 tracking with snapshots.

### Silver Layer Data Model
Your target silver layer model with surrogate keys and SCD2 tracking:

![Silver Layer Data Model](z_assets/silver-layer-model.png)

### Steps

#### 4.1 Add dbt-utils Package
Create `packages.yml` in project root (where `dbt_project.yml` is located):
```yaml
packages:
  - package: dbt-labs/dbt_utils
    version: 1.1.1
```

Install packages:
```bash
dbt deps
```

#### 4.2 Update Models with Surrogate Keys
Update `src/models/silver/customers.sql`:
```sql
{{ config(materialized='table') }}

SELECT 
    {{ dbt_utils.generate_surrogate_key(['c_custkey']) }} as customer_key,
    c_custkey as customer_id,
    c_name as customer_name,
    c_address as customer_address,
    {{ dbt_utils.generate_surrogate_key(['c_nationkey']) }} as nation_key,
    c_nationkey as nation_id,
    c_phone as customer_phone,
    c_acctbal as account_balance,
    c_mktsegment as market_segment,
    c_comment as customer_comment
FROM {{ source('bronze', 'customer') }}
```

Update all other silver models to include surrogate keys for all ID fields.

#### 4.3 Create Snapshots
Create `src/snapshots/customers_snapshot.sql`:
```sql
{% snapshot customers_snapshot %}

{{
    config(
      target_schema='<YOUR INITIALS>_silver_snapshots',
      unique_key='customer_id',
      strategy='check',
      check_cols='all'
    )
}}

SELECT * FROM {{ ref('customers') }}

{% endsnapshot %}
```

Create snapshots for all dimensional tables (customers, orders, parts, suppliers, nations, regions).

You don't need to create a snaptshot for `lineitems`, as this table will be used by our fact table. In practice, these types of table don't require SCD2 because:
- Transactions are typically immutable (no change)
- Massive data volume (performance)
- Don't need tracking because only the point-in-time is interesting

Contrary to models, the snapshot name is not defined by the filename. The name for the snapshot is specified within the file in the form of `{% snapshot customers_snapshot %}`. For consistency's sake, name the snapshot file the same as the name speficied within the file.

#### 4.4 Run Updated Models
```bash
# Run silver models
dbt run --select silver

# Run snapshots
dbt snapshot

# Combined run for model and snapshots
dbt build

# Wait for the changes in the bronze layer before re-run
dbt snapshot
```

Check for changes in Databricks SQL Editor.
```sql
-- Check for the changes (replace ek with your initials)
SELECT * FROM dbt_workshop.ek_silver_snapshots.customers_snapshot
WHERE customer_id IN (412445, 412446, 412447, 412448)
ORDER BY customer_id, dbt_valid_from;
```

### Success Criteria
- All models include surrogate keys
- Snapshots are created successfully
- SCD2 tracking is enabled (check dbt_valid_from, dbt_valid_to columns)

## Exercise 5: Gold Layer - Dimensions

### Goal
Create dimensional model with clean dimension tables.

### Gold Layer Data Model
Your target star schema for analytics:

![Gold Layer Data Model](z_assets/gold-layer-model.png)

### Steps

#### 5.1 Add configuration for gold layer in dbt_project.yml
Add information for gold layer in `dbt_project.yml`. The affected section should look like this:
```yml
models:
  dbt_workshop:
    silver:
      schema: silver
      +materialized: table
    gold:
      schema: gold
      +materialized: table
```

#### 5.2 Configure Gold Models
Create `src/models/gold/schema.yml`:
```yaml
version: 2

models:
  - name: dim_customer
    description: 'Customer dimension with geography'
    columns:
      - name: customer_key
        description: 'Surrogate key for customer'
        tests:
          - unique
          - not_null
      - name: customer_id
        description: 'Natural key for customer'
        tests:
          - unique
          - not_null
  
  - name: dim_supplier
    description: 'Supplier dimension with geography'
  
  - name: dim_part
    description: 'Part dimension'
  
  - name: fact_lineitem
    description: 'Line item fact table'
```

#### 5.3 Create Customer Dimension
Create `src/models/gold/dim_customer.sql`:
```sql
{{ config(materialized='table') }}

SELECT 
    c.customer_key,
    c.customer_id,
    c.customer_name,
    c.customer_address,
    c.customer_phone,
    c.account_balance,
    c.market_segment,
    n.nation_name,
    r.region_name
FROM {{ ref('customers_snapshot') }} c
JOIN {{ ref('nations_snapshot') }} n ON c.nation_key = n.nation_key
JOIN {{ ref('regions_snapshot') }} r ON n.region_key = r.region_key
WHERE c.dbt_valid_to IS NULL  -- Current records only
  AND n.dbt_valid_to IS NULL
  AND r.dbt_valid_to IS NULL
```

#### 5.4 Create Supplier Dimension
Create `src/models/gold/dim_supplier.sql`:
```sql
{{ config(materialized='table') }}

SELECT 
    s.supplier_key,
    s.supplier_id,
    s.supplier_name,
    s.supplier_address,
    s.supplier_phone,
    s.account_balance,
    n.nation_name,
    r.region_name
FROM {{ ref('suppliers_snapshot') }} s
JOIN {{ ref('nations_snapshot') }} n ON s.nation_key = n.nation_key
JOIN {{ ref('regions_snapshot') }} r ON n.region_key = r.region_key
WHERE s.dbt_valid_to IS NULL
  AND n.dbt_valid_to IS NULL
  AND r.dbt_valid_to IS NULL
```

#### 5.5 Create Part Dimension
Create `src/models/gold/dim_part.sql`:
```sql
{{ config(materialized='table') }}

SELECT 
    part_key,
    part_id,
    part_name,
    manufacturer,
    brand,
    part_type,
    part_size,
    container,
    retail_price
FROM {{ ref('parts_snapshot') }}
WHERE dbt_valid_to IS NULL
```

#### 5.6 Run Dimension Models
```bash
dbt run --select gold
```

#### 5.7 Create Date Dimension

For dim_date, we will use seeds. In essence, seeds in dbt is when you upload manual data into the database platform. To do that add the following line add the end of `dbt_project.yml`:

```yml
seeds:
  dbt_workshop:
    schema: gold
```

This line will specifies the target schema to create the table for the seed file.

Next, copy the file `dim_date.csv` from `workshop/seed-file` to `src/seeds`. The file name will also define the table name when loading the seed.

Finally, load the seed into Databricks with this command (might take up to 5 minutes):
```bash
dbt seed
```

Now, the table should be available in Databricks under the schema `<YOUR INITIALS>_gold`.

Note: When you run `dbt build`, the process will execute models, snapshots, and seeds. As the seed takes a while to load, please move the seed back to the `seed-file` directory or delete it, to avoid long processing times during the workshop.

### Success Criteria
- All dimension tables created successfully
- Joins between dimensions work correctly
- Only current records (dbt_valid_to IS NULL) are included

## Exercise 6: Gold Layer - Fact Table

### Goal
Create an incremental fact table for line items.

Note: The logic is very flawed and should just showcase the feature for incremental loads. It does not consider changes in lineitems, such as updated dates, etc. It does only look for new orders and append them in the fact table.

### Steps

#### 6.1 Create Fact Table
Create `src/models/gold/fact_lineitem.sql`:
```sql
{{
    config(
        materialized='incremental',
        unique_key='order_key || lineitem_key',
        incremental_strategy='append',
        on_schema_change='fail'
    )
}}

SELECT
    l.lineitem_key,
    o.order_date,
    l.line_number,
    -- Foreign keys to dimensions
    o.customer_key,
    l.part_key,
    l.supplier_key,
    l.quantity,

    -- Measures
    l.extended_price,
    l.discount,
    l.tax,
    l.return_flag,
    l.line_status,

    -- Shipping information and dates formatted for dim_date
    date_format(l.ship_date, 'yyyyMMdd') as ship_date,
    date_format(l.commit_date, 'yyyyMMdd') as commit_date,
    date_format(l.receipt_date, 'yyyyMMdd') as receipt_date,
    l.ship_instructions,
    l.ship_mode,
    
    -- Calculated measures
    l.extended_price * (1 - l.discount) as discounted_price,
    l.extended_price * (1 - l.discount) * (1 + l.tax) as total_price,
    
    -- Audit fields
    o.dbt_updated_at
    
FROM {{ ref('orders_snapshot') }} o
JOIN {{ ref('lineitems') }} l ON o.order_key = l.order_key
WHERE o.dbt_valid_to IS NULL

{% if is_incremental() %}
    -- Only process new orders
    AND o.order_date > (SELECT max(order_date) FROM {{ this }})
{% endif %}
```

#### 6.2 Run Fact Table
```bash
# Initial full load
dbt run --select gold.fact_lineitem

# Optional: Subsequent incremental runs - you will notice that the run is faster, as we do not have any new data to load
dbt run --select gold.fact_lineitem
```

#### 6.3 Test Your Star Schema
Run this query to verify your dimensional model:
```sql

-- Replace 'ek' with your initials.

SELECT
    d.date,
    d.quarter,
    c.customer_name,
    c.nation_name,
    p.part_name,
    s.supplier_name,
    f.quantity,
    f.total_price
FROM ek_gold.fact_lineitem f
JOIN ek_gold.dim_customer c ON f.customer_key = c.customer_key
JOIN ek_gold.dim_part p ON f.part_key = p.part_key
JOIN ek_gold.dim_supplier s ON f.supplier_key = s.supplier_key
JOIN ek_gold.dim_date d ON f.ship_date = d.date_key
LIMIT 10;
```

### Success Criteria
- Fact table loads successfully
- Incremental logic works (run twice to test)
- Star schema query returns meaningful results
- All foreign key relationships are intact

## Exercise 7: Testing & Documentation

### Goal
Add data quality tests and generate documentation.

### Steps

#### 7.1 Run Tests
```bash
dbt test
```

#### 7.2 Generate Documentation
```bash
dbt docs generate
dbt docs serve
```

#### 7.3 View Lineage
In the docs interface, explore:
- Data lineage graph
- Model documentation
- Test results

### Success Criteria
- All tests pass
- Documentation site loads
- Lineage graph shows bronze → silver → gold flow

## Final Validation

### Check Your Work
Run these commands to validate your complete solution:

```bash
# Run everything
dbt build

# Check row counts
dbt run-operation print_relation_row_counts
```

### Results
You should have:
- 8 bronze source tables
- 7 silver models + snapshots
- 4 gold dimensions + 1 fact table
- SCD2 tracking with dbt_valid_from/to
- Surrogate keys throughout
- Incremental fact table
- Star schema for analytics

### What You've Learned
- dbt project setup and configuration
- Source and model definitions
- Data transformation and cleansing
- Surrogate key generation
- SCD2 implementation with snapshots
- Dimensional modeling
- Incremental models for performance
- Testing and documentation